In [1]:
!sudo pip3 install --upgrade pip keras==2.1.5 tensorflow==1.13.1 numpy pandas pillow sklearn opencv-python optuna scikit-image  optkeras h5py==2.10.0

     |████████████████████████████████| 1.5 MB 1.5 MB/s eta 0:00:01
     |████████████████████████████████| 334 kB 2.2 MB/s eta 0:00:01
     |████████████████████████████████| 92.5 MB 442 kB/s eta 0:00:01    |▎                               | 921 kB 359 kB/s eta 0:04:15     |█                               | 3.0 MB 370 kB/s eta 0:04:02     |████▍                           | 12.7 MB 405 kB/s eta 0:03:17     |███████▍                        | 21.3 MB 418 kB/s eta 0:02:51     |███████▌                        | 21.6 MB 418 kB/s eta 0:02:50     |███████▋                        | 21.9 MB 418 kB/s eta 0:02:49     |█████████▍                      | 27.2 MB 424 kB/s eta 0:02:34     |███████████████████████▎        | 67.5 MB 414 kB/s eta 0:01:01
     |████████████████████████████████| 14.8 MB 34.4 MB/s eta 0:00:01
     |████████████████████████████████| 9.5 MB 20.1 MB/s eta 0:00:01
     |████████████████████████████████| 2.2 MB 1.8 MB/s eta 0:00:01
     |████████████████████████████████| 50.4 MB

     |████████████████████████████████| 1.1 MB 8.6 MB/s eta 0:00:01
Using legacy 'setup.py install' for sklearn, since package 'wheel' is not installed.
Using legacy 'setup.py install' for termcolor, since package 'wheel' is not installed.
Using legacy 'setup.py install' for Mako, since package 'wheel' is not installed.
Using legacy 'setup.py install' for PrettyTable, since package 'wheel' is not installed.
Using legacy 'setup.py install' for pyperclip, since package 'wheel' is not installed.
  Attempting uninstall: pip
    Found existing installation: pip 20.2.2
    Uninstalling pip-20.2.2:
      Successfully uninstalled pip-20.2.2
    Running setup.py install for termcolor ... done
    Running setup.py install for sklearn ... done
    Running setup.py install for Mako ... done
    Running setup.py install for pyperclip ... done
    Running setup.py install for PrettyTable ... done


In [2]:
!sudo pip3 install opencv-python imblearn

     |████████████████████████████████| 167 kB 3.2 MB/s eta 0:00:01


In [8]:
#!/usr/bin/env python3
import os
import logging
from pathlib import Path
import requests 
from glob import glob
from zipfile import ZipFile
import pickle
import pandas as pd

logging.basicConfig(level=logging.DEBUG)

#Import Pegasus API
from Pegasus.api import *
props = Properties()
props["pegasus.mode"] = "development"
props.write()

#Replica Catalog
rc = ReplicaCatalog()
in_files=[]

label_csv = './inputs/training_solutions_rev1.csv'
    
# for file in input_files:
#     in_files.append(File(file))
#     rc.add_replica("local", File(file), str(Path(".").resolve() / file))  
# rc.add_replica("local", File(label_csv), str(Path(".").resolve() / label_csv))
# rc.write()

for file in glob("./inputs/*.jpg"):
    f = file.replace("./inputs/", '')
    in_files.append(File(f))
    rc.add_replica("local", File(f), Path("./inputs/").resolve() / f)
    
csv = label_csv.replace("./inputs/",'')
rc.add_replica("local", File(csv), Path("./inputs/").resolve() / csv)
rc.write()
#Transformation

data_label = Transformation( "Label_Data.py",
            site="local",
            pfn="/home/scitech/shared-data/Galaxy2/Label_Data.py",
            is_stageable=True
            )

crop_images = Transformation( "Image_Crop.py",
            site="local",
            pfn="/home/scitech/shared-data/Galaxy2/Image_Crop.py",
            is_stageable=True
            )

oversampling = Transformation( "Over_Sampling.py",
            site="local",
            pfn="/home/scitech/shared-data/Galaxy2/Over_Sampling.py",
            is_stageable=True
            )

tc = TransformationCatalog()\
    .add_transformations(data_label,crop_images,oversampling)\
    .write()

#Workflow
wf = Workflow("Galaxy", infer_dependencies=True)

dataset = []
for i in range(8):
    dataset.append(File('Class0_'+ str(i)+'.jpg'))
for i in range(8):
    dataset.append(File('Class1_'+ str(i)+'.jpg'))
for i in range(3):
    dataset.append(File('Class2_'+ str(i)+'.jpg'))
for i in range(4):
    dataset.append(File('Class3_'+ str(i)+'.jpg'))
for i in range(6):
    dataset.append(File('Class4_'+ str(i)+'.jpg'))
    
job_data_label = Job(data_label)\
                    .add_inputs(*in_files, File(csv))\
                    .add_outputs(*dataset)

resized_images = [File("resized_{}".format(f.lfn)) for f in dataset]
job_resize = Job(crop_images)\
                    .add_inputs(*dataset)\
                    .add_outputs(*resized_images) 

Train_csv = File('Training_Data.csv')
Test_csv = File('Test_Data.csv')

job_oversampling = Job(oversampling)\
                    .add_inputs(*resized_images)\
                    .add_outputs(Train_csv, Test_csv)

wf.add_jobs(job_data_label, job_resize,job_oversampling)                                    

In [9]:
try:
     wf.plan(submit=True)\
        .wait()\
        .analyze()\
        .statistics()
except PegasusClientError as e:
    print(e.output)


################
# pegasus-plan #
################
[main] WARN  schema.JsonMetaSchema  - Unknown keyword $defs - you should define your own Meta Schema. If the keyword is irrelevant for validation, just use a NonValidationKeyword
[main] WARN  schema.JsonMetaSchema  - Unknown keyword additionalItems - you should define your own Meta Schema. If the keyword is irrelevant for validation, just use a NonValidationKeyword
[main] WARN  schema.JsonMetaSchema  - Unknown keyword examples - you should define your own Meta Schema. If the keyword is irrelevant for validation, just use a NonValidationKeyword
2021.01.29 19:14:39.674 UTC:
2021.01.29 19:14:39.680 UTC:   -----------------------------------------------------------------------
2021.01.29 19:14:39.685 UTC:   File for submitting this DAG to HTCondor           : Galaxy-0.dag.condor.sub
2021.01.29 19:14:39.691 UTC:   Log of DAGMan debugging messages                 : Galaxy-0.dag.dagman.out
2021.01.29 19:14:39.696 UTC:   Log of HTCondor libra

[##################################################] 100.0% ..Success (Completed: 19, Queued: 0, Running: 0, Failed: 0)



####################
# pegasus-analyzer #
####################
Your database is compatible with Pegasus version: 5.0.0dev

************************************Summary*************************************

Submit Directory   : /home/scitech/shared-data/Galaxy2/scitech/pegasus/Galaxy/run0003
Total jobs         :     19 (100.00%)
# jobs succeeded   :     19 (100.00%)
# jobs failed      :      0 (0.00%)
# jobs held        :      0 (0.00%)
# jobs unsubmitted :      0 (0.00%)



######################
# pegasus-statistics #
######################
Your database is compatible with Pegasus version: 5.0.0dev

#
# Pegasus Workflow Management System - http://pegasus.isi.edu
#
# Workflow summary:
#   Summary of the workflow execution. It shows total
#   tasks/jobs/sub workflows run, how many succeeded/failed etc.
#   In case of hierarchical workflow the calculation shows the
#   statistics across all the sub workflows.It shows the following
#   statistics about tasks, jobs and sub workflows.
#    